# Facial Emotion Recognition

## importing libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime
import matplotlib.pyplot as plt
from keras.utils import plot_model
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.image import load_img,img_to_array
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [ ]:
# RESNET
from keras.applications.resnet import ResNet101, ResNet152, ResNet50

# VGG
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
#from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.applications.vgg19 import VGG19

# INCEPTION
from keras.applications.inception_v3 import InceptionV3
# DENSENET
#from tensorflow.keras.applications.denset import InceptionV3
#from tensorflow.keras.applications.inception_v3 import InceptionV3
# NASNET
# XCEPTION
# EFFICIENTNET
# MOBILENET

from keras.layers import Input, Dense, GlobalAveragePooling2D

## Importing Dataset

In [ ]:
train_dir = '../input/fer2013/train/'
test_dir = '../input/fer2013/test/'

row, col = 48, 48
classes = 7

def count_exp(path, set_):
    dict_ = {}
    for expression in os.listdir(path):
        dir_ = path + expression
        dict_[expression] = len(os.listdir(dir_))
    df = pd.DataFrame(dict_, index=[set_])
    return df
train_count = count_exp(train_dir, 'train')
test_count = count_exp(test_dir, 'test')
print(train_count)
print(test_count)

### PLot of number of images in training set

In [ ]:
train_count.transpose().plot(kind='bar')

### PLot of number of images in test set

In [ ]:
test_count.transpose().plot(kind='bar')

In [ ]:
plt.figure(figsize=(14,22))
i = 1
for expression in os.listdir(train_dir):
    img = load_img((train_dir + expression +'/'+ os.listdir(train_dir + expression)[1]))
    plt.subplot(1,7,i)
    plt.imshow(img)
    plt.title(expression)
    plt.axis('off')
    i += 1
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
cpt=0
for expression in os.listdir(train_dir):
    for i in range(1,8):
        cpt += 1
        plt.subplot(7,8,cpt)
        img=load_img(train_dir+expression+"/"+os.listdir(train_dir+expression)[i],target_size=(48,48))
        plt.imshow(img,cmap='gray')
        #plt.xlabel(os.listdir(train_dir+expression)[i])
        plt.title(expression)
plt.tight_layout()
plt.show()

In [ ]:
TOP_EMOTIONS = ["surprise", "fear", "angry", "neutral", "sad", "disgust","happy"]
total_images

## Vision Transformer

In [ ]:
TRAIN_PATH = "../input/fer2013/train/"
TEST_PATH = "../input/fer2013/test/"

In [ ]:
total_images = 0
for dir_ in os.listdir(TRAIN_PATH):
    count = 0
    for f in os.listdir(TRAIN_PATH + dir_ + "/"):
        count += 1
        total_images += 1
    print(f"{dir_} has {count} number of images")
    
print(f"\ntotal images are {total_images}")

In [ ]:
img_arr = np.empty(shape=(total_images,48,48,3))
img_label = np.empty(shape=(total_images))
label_to_text = {}

i = 0
e = 0
for dir_ in os.listdir(TRAIN_PATH):
    if dir_ in TOP_EMOTIONS:
        label_to_text[e] = dir_
        for f in os.listdir(TRAIN_PATH + dir_ + "/"):
            img_arr[i] = cv2.imread(TRAIN_PATH + dir_ + "/" + f)
            img_label[i] = e
            i += 1
        print(f"loaded all {dir_} images to numpy arrays")
        e += 1

img_arr.shape, img_label

In [ ]:
img_label = np_utils.to_categorical(img_label)
img_label.shape

In [ ]:
img_arr = img_arr / 255.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(img_arr, img_label,
                                                    shuffle=True, stratify=img_label,
                                                    train_size=0.9, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
num_classes = 7
input_shape = (48,48, 3)

#(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

#print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
#print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 30
image_size = 72  # We'll resize input images to this size
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        layers.experimental.preprocessing.Resizing(image_size, image_size),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(factor=0.02),
        layers.experimental.preprocessing.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.


In [ ]:
data_augmentation.layers[0].adapt(x_train)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
image = x_train[np.random.choice(range(x_train.shape[0]))]
plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(image_size, image_size)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    checkpoint_filepath = "/tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_split=0.1,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


In [ ]:
!pip install -U tensorflow==2.4.0

In [ ]:
tf.__version__

In [ ]:
vit_classifier = create_vit_classifier()
history = run_experiment(vit_classifier)

## Creating Training and test sets

### Pre-trained Model

In [ ]:

IMAGE_SIZE=[224,224]
train_path='../input/fer2013/train'

valid_path='../input/fer2013/test'

folders= glob.glob('../input/fer2013/train/*')
folders

In [ ]:
res50 = ResNet50(weights='imagenet',input_shape=IMAGE_SIZE+[3],include_top=False)

for layer in res50.layers:
    layer.trainable=False
x= Flatten()(res50.output)

prediction = Dense(len(folders),activation='softmax')(x)

model_res50 = Model(inputs=res50.input,outputs=prediction)
model_res50.summary()

## Callbacks

In [ ]:
chk_path = 'resnet50.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             mode='min',
                             moniter='val_loss')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

## Training

In [ ]:

#model_res50.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=[0.8,1.2],
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(224,224),
                                                 batch_size=128,
                                                 #shuffle=True,
                                                 #color_mode="grayscale",
                                                 class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size=(224,224),
                                            batch_size=128,
                                            #shuffle=False,
                                            #color_mode="grayscale",
                                            class_mode='categorical')
training_set.class_indices

In [ ]:
%%time

steps_per_epoch = len(training_set) // 128
validation_steps = len(test_set) // 128

hist_r50 = model_res50.fit_generator(training_set,
                 validation_data=test_set
                 epochs=60,
                 callbacks=callbacks,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

In [ ]:
#history_r50 = model_res50.fit_generator(training_set,
 #                                       validation_data=test_set,
  #                                      epochs=30,
   #                                     steps_per_epoch=len(training_set),
    #                                    validation_steps=len(test_set))

## Defining Model

In [ ]:
def get_model(input_size, classes=7):
     #Initialising the CNN
    model = tf.keras.models.Sequential()   

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape =input_size))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(classes, activation='softmax'))

    #Compliling the model
    model.compile(optimizer=Adam(lr=0.0001, decay=1e-6), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [ ]:
fernet = get_model((row,col,1), classes)
fernet.summary()

In [ ]:
plot_model(fernet, to_file='fernet.png', show_shapes=True, show_layer_names=True)

In [ ]:
%%time

history1 = fernet.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = 50,
                    callbacks = [callbacks])

### NASNet

In [ ]:
import keras

In [ ]:
nasnet = keras.applications.NASNetMobile(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet")

In [ ]:
for layer in nasnet.layers:
    layer.trainable=False
x= Flatten()(nasnet.output)

prediction = Dense(len(folders),activation='softmax')(x)

model_nasnet = Model(inputs=nasnet.input,outputs=prediction)
model_nasnet.summary()

In [ ]:

fernet.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=[0.8,1.2],
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(224,224),
                                                 batch_size=128,
                                                 #shuffle=True,
                                                 #color_mode="grayscale",
                                                 class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size=(224,224),
                                            batch_size=128,
                                            #shuffle=False,
                                            #color_mode="grayscale",
                                            class_mode='categorical')
training_set.class_indices

In [ ]:
%%time

#steps_per_epoch = len(training_set) // 128
#validation_steps = len(test_set) // 128

hist_fernet = fernet.fit_generator(training_set,
                 validation_data=test_set,
                 #verbose=1,
                 epochs=60,
                 callbacks=callbacks,
                 #steps_per_epoch=steps_per_epoch,
                 validation_steps=test_set/256)

In [ ]:
import tensorflow as tf

## EfficientNet B2

In [ ]:
effnetb2 = tf.keras.applications.EfficientNetB2(
    include_top=False,
    #weights="imagenet",
    input_shape=(48,48,3),
    classes=7
)

for layer in effnetb2.layers:
    layer.trainable=False
x= Flatten()(effnetb2.output)

prediction = Dense(len(folders),activation='softmax')(x)

model_effnetb2 = Model(inputs=effnetb2.input,outputs=prediction)
model_effnetb2.summary()

In [ ]:

model_effnetb2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=[0.8,1.2],
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(48,48),
                                                 batch_size=128,
                                                 #shuffle=True,
                                                 #color_mode="grayscale",
                                                 class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size=(48,48),
                                            batch_size=128,
                                            #shuffle=False,
                                            #color_mode="grayscale",
                                            class_mode='categorical')
training_set.class_indices

In [ ]:
chk_path = 'effnetb2.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             mode='min',
                             moniter='val_loss')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

In [ ]:
%%time

steps_per_epoch = len(training_set) // 128
validation_steps = len(test_set) // 128

hist_effnetb2 = model_effnetb2.fit_generator(training_set,
                 validation_data=test_set,
                 verbose=1,
                 epochs=60,
                 callbacks=callbacks,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

### REDO

## EfficientNetB2

In [ ]:
pre_model = tf.keras.applications.EfficientNetB2(
    include_top=False,
    #weights="imagenet",
    input_shape=(48,48,3),
    classes=7
)


In [ ]:
for layer in pre_model.layers:
    layer.trainable=False
last_output= (pre_model.output)


In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Dense(32,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)
prediction = Dense(len(folders),activation='softmax')(x)

model = Model(inputs=pre_model.input,outputs=prediction)
#model.summary()

model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
%%time
#steps_per_epoch = len(train_generator) // 256
#validation_steps = len(test_set) // 256

history = model.fit(train_generator,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e1,
                   #steps_per_epoch=steps_per_epoch,
                  )

In [ ]:
%%time
for layer in model.layers:
    layer.trainable = True
#print(model.summary())
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1_effnet = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e2,
                    callbacks = [callbacks])

In [ ]:
N=e2
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history1_effnet.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history1_effnet.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history1_effnet.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history1_effnet.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on EfficientNetB2")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("effnetb2.png")

## DenseNet-121

In [ ]:
pre_model = tf.keras.applications.DenseNet121(
    include_top=False,
    #weights="imagenet",
    input_shape=(224,224,3),
    classes=7
)
for layer in pre_model.layers:
    layer.trainable=False
last_output= (pre_model.output)
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Dense(32,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)
prediction = Dense(len(folders),activation='softmax')(x)

model = Model(inputs=pre_model.input,outputs=prediction)
#model.summary()

model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
%%time
#steps_per_epoch = len(train_generator) // 256
#validation_steps = len(test_set) // 256

history = model.fit(train_generator,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e1,
                   #steps_per_epoch=steps_per_epoch,
                  )

In [ ]:
%%time
for layer in model.layers:
    layer.trainable = True
#print(model.summary())
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1_densenet = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e2,
                    callbacks = [callbacks])

In [ ]:
N=e2
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history1_effnet.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history1_effnet.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history1_effnet.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history1_effnet.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on DenseNet121")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("densenet121.png")

## InceptionV3

In [ ]:
pre_model = tf.keras.applications.InceptionV3(
    include_top=False,
    #weights="imagenet",
    input_shape=(299, 299 ,3),
    classes=7
)


In [ ]:
for layer in pre_model.layers:
    layer.trainable=False
last_output= (pre_model.output)
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Dense(32,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)
prediction = Dense(len(folders),activation='softmax')(x)

model = Model(inputs=pre_model.input,outputs=prediction)
#model.summary()

model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
%%time
#steps_per_epoch = len(train_generator) // 256
#validation_steps = len(test_set) // 256

history = model.fit(train_generator,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e1,
                   #steps_per_epoch=steps_per_epoch,
                  )

In [ ]:
%%time
for layer in model.layers:
    layer.trainable = True
#print(model.summary())
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1_incv3 = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e2,
                    callbacks = [callbacks])

In [ ]:
N=e2
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history1_incv3.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history1_incv3.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history1_incv3.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history1_incv3.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Inception-V3")
plt.xlabel("Epochs")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("inceptionv3.png")

## VGG-16

In [ ]:
from keras.applications.vgg16 import VGG16
pre_model = VGG16(include_top = False,
                 weights = 'imagenet',
                 input_shape = (48,48,3))


In [ ]:
for layer in pre_model.layers:
    layer.trainable=False

In [ ]:
last_output = pre_model.output

In [ ]:
# from keras.backend import sigmoid
# def swish(x, beta = 1):
#     return (x * sigmoid(beta * x))

In [ ]:
# from keras.utils.generic_utils import get_custom_objects
# from keras.layers import Activation
# get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Dense(32,activation= 'swish')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)

In [ ]:
model = tf.keras.Model(pre_model.input, x)

In [ ]:
#Learning Rate and Epochs for Model Warm-up
lr1 = 1e-5
e1 = 15

#Learning Rate and Epochs for Model Training
lr2 = 1e-4
e2 = 30

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=[0.8,1.2],
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2
                                  )
                                  
val_datagen = ImageDataGenerator(rescale=1./255)                         
                                
#Batch Size and Image Size
batch_sz = 256
sz = 48

train_generator = train_datagen.flow_from_directory('../input/fer2013/train',
                                                    batch_size = batch_sz,
                                                    class_mode='categorical',
                                                    target_size=(sz,sz))

val_generator = val_datagen.flow_from_directory('../input/fer2013/test',
                                                    batch_size = batch_sz,
                                                    class_mode='categorical',
                                                    target_size=(sz,sz))

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.75):
#             if(logs.get('accuracy')>78):
                print("\nReached 74% val_accuracy, so cancelling training!")
                self.model.stop_training = True
callbacks = myCallback()

In [ ]:
#opt = tf.keras.optimizers.Adam(learning_rate = lr1)
opt = tf.keras.optimizers.Adam(learning_rate=lr1)

model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
%%time
#steps_per_epoch = len(train_generator) // 256
#validation_steps = len(test_set) // 256

history = model.fit(train_generator,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e1,
                   #steps_per_epoch=steps_per_epoch,
                  )

In [ ]:
e2=30

In [ ]:
%%time
for layer in model.layers:
    layer.trainable = True
#print(model.summary())
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1 = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e2,
                    callbacks = [callbacks])

In [ ]:
test_score = model.evaluate_generator(val_generator, 256)



print("[INFO] accuracy: {:.2f}%".format(test_score[1] * 100)) 

print("[INFO] Loss: ",test_score[0])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion matrix', cmap='inferno'):

    """

    This function prints and plots the confusion matrix.

    Normalization can be applied by setting `normalize=True`.

    """

    plt.figure(figsize=(10,10))



    plt.imshow(cm, interpolation='nearest', cmap=cmap)

    plt.title(title)

    plt.colorbar()



    tick_marks = np.arange(len(classes))

    plt.xticks(tick_marks, classes, rotation=45)

    plt.yticks(tick_marks, classes)



    if normalize:

        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        cm = np.around(cm, decimals=2)

        cm[np.isnan(cm)] = 0.0

        print("Normalized confusion matrix")

    else:

        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):

        plt.text(j, i, cm[i, j],

                 horizontalalignment="center",

                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()

    plt.ylabel('True label')

    plt.xlabel('Predicted label')



##################################################################################################





#Print the Target names



target_names = []

for key in train_generator.class_indices:

    target_names.append(key)



# print(target_names)



#Confution Matrix 



Y_pred = model.predict_generator(val_generator)

y_pred = np.argmax(Y_pred, axis=1)

print('Confusion Matrix')

cm = confusion_matrix(val_generator.classes, y_pred)

plot_confusion_matrix(cm, target_names, title='Confusion Matrix')



#Print Classification Report

print('Classification Report')

print(classification_report(val_generator.classes, y_pred, target_names=target_names))



#Save the model

model.save("tutorial.hdf5")





In [ ]:
model.save('vgg_fer.h5')

In [ ]:
model.save_weights('vgg_fer_weights.h5')

In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
 
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
 
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()
 
# Print out model inputs and outputs
print("Frozen model inputs: ", frozen_func.inputs)
print("Frozen model outputs: ", frozen_func.outputs)
 
# Save frozen graph to disk
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="VGG16_1e-5_10_1e-4_17(gap,32,4).pb",
                  as_text=False)

In [ ]:
N=e2
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history1.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history1.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history1.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history1.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on VGG-16")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("vgg16.png")

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

## ResNet50, EffB2, NASNetMobile

In [ ]:
pre_model = ResNet50(include_top = False,weights="imagenet",input_shape = (224,224,3))
for layer in pre_model.layers:
    layer.trainable=False
    

In [ ]:
last_output = pre_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Dense(32,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)
model = tf.keras.Model(pre_model.input, x)
#Learning Rate and Epochs for Model Warm-up
lr1 = 1e-5
e1 = 15

#Learning Rate and Epochs for Model Training
lr2 = 1e-4
e2 = 30

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.74):
#             if(logs.get('accuracy')>78):
                print("\nReached 74% val_accuracy, so cancelling training!")
                self.model.stop_training = True
callbacks = myCallback()

opt = tf.keras.optimizers.Adam(learning_rate = lr1)
model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
%%time
history_res = model.fit(train_generator,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e1)

In [ ]:
%%time
for layer in model.layers:
    layer.trainable = True
#print(model.summary())
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1_res = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e2,
                    callbacks = [callbacks])

In [ ]:
model.summary()

## MobileNet

In [ ]:
from keras.applications.vgg16 import VGG16
pre_model = tf.keras.applications.MobileNet(include_top = False,
                 weights = 'imagenet',
                 input_shape = (48,48,3))
for layer in pre_model.layers:
    layer.trainable=False
    


In [ ]:
from keras.applications.vgg16 import VGG16
pre_model = tf.keras.applications.Xception(include_top = False,
                 weights = 'imagenet',
                 input_shape = (71,71,3))
for layer in pre_model.layers:
    layer.trainable=False
    


In [ ]:
last_output = pre_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Dense(32,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)
model = tf.keras.Model(pre_model.input, x)
#Learning Rate and Epochs for Model Warm-up
lr1 = 1e-5
e1 = 15

#Learning Rate and Epochs for Model Training
lr2 = 1e-4
e2 = 30

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.74):
#             if(logs.get('accuracy')>78):
                print("\nReached 74% val_accuracy, so cancelling training!")
                self.model.stop_training = True
callbacks = myCallback()

opt = tf.keras.optimizers.Adam(learning_rate = lr1)
model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
%%time
history_xception = model.fit(train_generator,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e1)

In [ ]:
%%time
for layer in model.layers:
    layer.trainable = True
#print(model.summary())
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1_xception = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e2,
                    callbacks = [callbacks])

In [ ]:
N=e2
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history1_xception.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history1_xception.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history1_xception.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history1_xception.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Xception")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("xception.png")

In [ ]:
N=e2
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history1_res.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history1_res.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history1_res.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history1_res.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on ResNet50")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("res50.png")

In [ ]:
N=e2
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history1.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history1.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history1.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history1.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on VGG-16")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")
plt.savefig("vgg16.png")

In [ ]:
model.summary()

In [ ]:
sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(history1_xception.epoch, history1_xception.history['accuracy'], label='train')
sns.lineplot(history1_xception.epoch, history1_xception.history['val_accuracy'], label='valid')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(history1_xception.epoch, history1_xception.history['loss'], label='train')
sns.lineplot(history1_xception.epoch, history1_xception.history['val_loss'], label='valid')
plt.title('Loss')
plt.tight_layout()

plt.savefig('epoch_history_resnet50.png')
plt.show()

In [ ]:
import matplotlib.pyplot as pyplot
df_accu = pd.DataFrame({'train': history1_xception.history['accuracy'], 'valid': history1_xception.history['val_accuracy']})
df_loss = pd.DataFrame({'train': history1_xception.history['loss'], 'valid': history1_xception.history['val_loss']})

fig = pyplot.figure(0, (14, 4))
ax = pyplot.subplot(1, 2, 1)
sns.violinplot(x="variable", y="value", data=pd.melt(df_accu), showfliers=False)
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.violinplot(x="variable", y="value", data=pd.melt(df_loss), showfliers=False)
pyplot.title('Loss')
pyplot.tight_layout()

pyplot.savefig('performance_vgg16.png')
pyplot.show()

## Redoing VGG

In [ ]:
from keras.applications.vgg16 import VGG16
pre_model = tf.keras.applications.VGG16(include_top = False,
                 weights = 'imagenet',
                 input_shape = (48,48,3))
for layer in pre_model.layers:
    layer.trainable=False
    


In [ ]:
last_output = pre_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Dense(32,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)
model = tf.keras.Model(pre_model.input, x)
#Learning Rate and Epochs for Model Warm-up
lr1 = 1e-5
e1 = 15

#Learning Rate and Epochs for Model Training
lr2 = 1e-4
e2 = 30

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.74):
#             if(logs.get('accuracy')>78):
                print("\nReached 74% val_accuracy, so cancelling training!")
                self.model.stop_training = True
callbacks = myCallback()

opt = tf.keras.optimizers.Adam(learning_rate = lr1)
model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
%%time
history_vgg16 = model.fit(train_generator,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e1)

In [ ]:
%%time
for layer in model.layers:
    layer.trainable = True
#print(model.summary())
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1_vgg16 = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e2,
                    callbacks = [callbacks])

In [ ]:
sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(history1_vgg16.epoch, history1_vgg16.history['accuracy'], label='train')
sns.lineplot(history1_vgg16.epoch, history1_vgg16.history['val_accuracy'], label='valid')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(history1_vgg16.epoch, history1_vgg16.history['loss'], label='train')
sns.lineplot(history1_vgg16.epoch, history1_vgg16.history['val_loss'], label='valid')
plt.title('Loss')
plt.tight_layout()

plt.savefig('epoch_history_resnet50.png')
plt.show()

In [ ]:
import matplotlib.pyplot as pyplot
df_accu = pd.DataFrame({'train': history1_vgg16.history['accuracy'], 'valid': history1_vgg16.history['val_accuracy']})
df_loss = pd.DataFrame({'train': history1_vgg16.history['loss'], 'valid': history1_vgg16.history['val_loss']})

fig = pyplot.figure(0, (14, 4))
ax = pyplot.subplot(1, 2, 1)
sns.violinplot(x="variable", y="value", data=pd.melt(df_accu), showfliers=False)
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.violinplot(x="variable", y="value", data=pd.melt(df_loss), showfliers=False)
pyplot.title('Loss')
pyplot.tight_layout()

pyplot.savefig('performance_vgg16.png')
pyplot.show()

In [ ]:
model.summary()

In [ ]:
last_output = pre_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(last_output)
x = tf.keras.layers.Dense(32,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)
model = tf.keras.Model(pre_model.input, x)
#Learning Rate and Epochs for Model Warm-up
lr1 = 1e-5
e1 = 15

#Learning Rate and Epochs for Model Training
lr2 = 1e-4
e2 = 30

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.74):
#             if(logs.get('accuracy')>78):
                print("\nReached 74% val_accuracy, so cancelling training!")
                self.model.stop_training = True
callbacks = myCallback()

opt = tf.keras.optimizers.Adam(learning_rate = lr1)
model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history_mobile = model.fit(train_generator,
                   validation_data = val_generator,
                   validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e1)

In [ ]:
%%time
for layer in model.layers:
    layer.trainable = True
#print(model.summary())
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

history1_mobile = model.fit(train_generator,
                    validation_data = val_generator,
                    validation_steps = val_generator.samples // val_generator.batch_size,
                    epochs = e2,
                    callbacks = [callbacks])

In [ ]:
model.summary()

In [ ]:
sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(history1_mobile.epoch, history1_mobile.history['accuracy'], label='train')
sns.lineplot(history1_mobile.epoch, history1_mobile.history['val_accuracy'], label='valid')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(history1_mobile.epoch, history1_mobile.history['loss'], label='train')
sns.lineplot(history1_mobile.epoch, history1_mobile.history['val_loss'], label='valid')
plt.title('Loss')
plt.tight_layout()

plt.savefig('epoch_history_resnet50.png')
plt.show()

In [ ]:
import matplotlib.pyplot as pyplot
df_accu = pd.DataFrame({'train': history1_mobile.history['accuracy'], 'valid': history1_mobile.history['val_accuracy']})
df_loss = pd.DataFrame({'train': history1_mobile.history['loss'], 'valid': history1_mobile.history['val_loss']})

fig = pyplot.figure(0, (14, 4))
ax = pyplot.subplot(1, 2, 1)
sns.violinplot(x="variable", y="value", data=pd.melt(df_accu), showfliers=False)
pyplot.title('Accuracy')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.violinplot(x="variable", y="value", data=pd.melt(df_loss), showfliers=False)
pyplot.title('Loss')
pyplot.tight_layout()

pyplot.savefig('performance_vgg16.png')
pyplot.show()

In [ ]:
mapper = {
    0: "surprise",
    1: "fear",
    2: "angry",
    3: "neutral",
    4: "sad",
    5: "disgust",
    6: "happy"
}

np.random.seed(2)
random_sad_imgs = np.random.choice(np.where(y_valid[:, 1]==1)[0], size=9)
random_neutral_imgs = np.random.choice(np.where(y_valid[:, 2]==1)[0], size=9)

fig = pyplot.figure(1, (18, 4))

for i, (sadidx, neuidx) in enumerate(zip(random_sad_imgs, random_neutral_imgs)):
        ax = pyplot.subplot(2, 9, i+1)
        sample_img = X_valid[sadidx,:,:,0]
        ax.imshow(sample_img, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"true:sad, pred:{mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")

        ax = pyplot.subplot(2, 9, i+10)
        sample_img = X_valid[neuidx,:,:,0]
        ax.imshow(sample_img, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"t:neut, p:{mapper[model.predict_classes(sample_img.reshape(1,48,48,1))[0]]}")

        pyplot.tight_layout()

### Xception

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255.,
                                   rotation_range=30,
                                   horizontal_flip=True,
                                   width_shift_range=0.15,
                                   height_shift_range=0.15,
                                   #shear_range=0.15,
                                   zoom_range=0.15,
                                   zca_whitening=False,
#                                    shear_range =20 ,
                                   brightness_range = [0.8,1.2]
                                )
                                  
val_datagen = ImageDataGenerator(rescale=1./255,
#                                 shear_range=20
                                )
#Batch Size and Image Size
batch_sz = 256
sz = 48

train_generator = train_datagen.flow_from_directory('../input/fer2013/train',
                                                    batch_size = batch_sz,
                                                    class_mode='categorical',
                                                    target_size=(sz,sz))

val_generator = val_datagen.flow_from_directory('../input/fer2013/test',
                                                    batch_size = batch_sz,
                                                    class_mode='categorical',
                                                    target_size=(sz,sz))

In [ ]:
NUM_CLASSES = 7


def xception_model():
    
    model=  tf.keras.Sequential()
    
    model.add(tf.keras.applications.Xception(input_shape=(71, 71,3),
                                             include_top = False,
                                            weights="imagenet", classes=  NUM_CLASSES))
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(7, activation='softmax'))
    opt = tf.keras.optimizers.Adam(lr=0.0001 , decay=1e-6)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
xception = xception_model()
xception.summary()

In [ ]:
plot_model(xception, to_file='model_1.png', show_shapes=True, show_layer_names=True)

In [ ]:
chk_path = 'model_1.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             #verbose=1,
                             mode='min',
                             moniter='val_loss')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          #verbose=1, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              verbose=1, 
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]

In [ ]:
batch_size =256
epochs=35
history = xception.fit_generator(
    train_generator,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs=epochs,
    callbacks=callbacks,
    use_multiprocessing=False
)

In [ ]:
sns.set()
fig = plt.figure(0, (12, 4))

ax = plt.subplot(1, 2, 1)
sns.lineplot(history1.epoch, history1.history['accuracy'], label='train')
sns.lineplot(history1.epoch, history1.history['val_accuracy'], label='valid')
plt.title('Accuracy')
plt.tight_layout()

ax = plt.subplot(1, 2, 2)
sns.lineplot(history1.epoch, history1.history['loss'], label='train')
sns.lineplot(history1.epoch, history1.history['val_loss'], label='valid')
plt.title('Loss')
plt.tight_layout()

plt.savefig('epoch_history_xception.png')
plt.show()

## Custom

In [ ]:


from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import numpy as np


chk_path = 'custom.h5'
log_dir = "checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             mode='min',
                             moniter='val_loss')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              min_delta=0.0001)


tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger('training.log')

callbacks = [checkpoint, reduce_lr, csv_logger]


base_model = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(48,48,3),
    classes=7
)

#model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)

#img_path = 'elephant.jpg'
#img = image.load_img(img_path, target_size=(224, 224))
#x = image.img_to_array(img)
#x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

#block4_pool_features = model.predict(x)


for layer in base_model.layers:
  layer.trainable=False

In [ ]:
#x= Flatten()(xcep.output)
base_output = base_model.output

In [ ]:
base_output.shape

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_output)

In [ ]:
x = tf.keras.layers.Dense(64, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = tf.keras.layers.LeakyReLU()(x)

In [ ]:
x.shape

In [ ]:
x = tf.keras.layers.Dropout(0.25)(x)

In [ ]:
x.shape

In [ ]:

x = tf.keras.layers.Dense(32,kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = tf.keras.layers.LeakyReLU()(x)

In [ ]:

x = tf.keras.layers.Dropout(0.25)(x)

In [ ]:

pred = tf.keras.layers.Dense(len(folders),activation='softmax',name = 'Output')(x)

In [ ]:
model = Model(inputs=base_model.input,outputs=pred)

In [ ]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=[0.8,1.2],
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(48,48),
                                                 batch_size=128,
                                                 #shuffle=True,
                                                 #color_mode="grayscale",
                                                 class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size=(48,48),
                                            batch_size=128,
                                            #shuffle=False,
                                            #color_mode="grayscale",
                                            class_mode='categorical')
training_set.class_indices

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_output)
x = tf.keras.layers.Dense(32,activation= 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
# x = tf.keras.layers.Dense(16,activation= 'relu')(x)
# x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(7,activation='softmax',name = 'Output')(x)

In [ ]:
model = tf.keras.Model(base_model.input, x)

In [ ]:
#Learning Rate and Epochs for Model Warm-up
lr1 = 1e-5
e1 = 10

#Learning Rate and Epochs for Model Training
lr2 = 1e-4
e2 = 17

opt = tf.keras.optimizers.Adam(learning_rate = lr1)
model.compile(optimizer = opt,loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
%%time

steps_per_epoch = len(training_set) // 128
validation_steps = len(test_set) // 128

hist1 = model.fit_generator(training_set,
                 validation_data=test_set,
                 verbose=1,
                 epochs=60,
                 callbacks=callbacks,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

In [ ]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
opt1 = tf.keras.optimizers.Adam(learning_rate = lr2)
model.compile(optimizer = opt1, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history1 = model.fit(training_set,
                    validation_data=test_set,                     
                    validation_steps =validation_steps,
                    epochs = e2,
                    steps_per_epoch=steps_per_epoch,
                    callbacks = callbacks)


In [ ]:
#x = tf.keras.layers.Dense(32, kernel_initializer='lecun_normal',activation='selu')(x)
#x = tf.keras.layers.Dropout(0.3)(x)
#pred = tf.keras.layers.Dense(len(folders),activation='softmax',name = 'Output')(x)
#kernel_initializer=tf.keras.initializers.GlorotNormal(),
#model = Model(inputs=base_output,outputs=pred)
#model.summary()

In [ ]:

model_xcep.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=[0.8,1.2],
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2
                                  )

test_datagen = ImageDataGenerator(rescale=1./255)


training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(48,48),
                                                 batch_size=128,
                                                 #shuffle=True,
                                                 #color_mode="grayscale",
                                                 class_mode='categorical')
test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size=(48,48),
                                            batch_size=128,
                                            #shuffle=False,
                                            #color_mode="grayscale",
                                            class_mode='categorical')
training_set.class_indices

In [ ]:
tf.keras.applications.xception.preprocess_input

In [ ]:
%%time

steps_per_epoch = len(training_set) // 128
validation_steps = len(test_set) // 128

hist_xcep = model_xcep.fit_generator(training_set,
                 validation_data=test_set,
                 epochs=60,
                 callbacks=callbacks,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

## Training Model

## Loss and Accuracy plot

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(history1.history['accuracy'])
#plt.plot(hist_r50.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(history1.history['loss'])
#plt.plot(hist_r50.history['val_loss'])
plt.title('model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

high accracy is achieved on training set but accuracy on validation set is stuck at 66% also no overfitting can se seen in the dataset hence is can be concluded that the inefficiency may be due to the unbalanced dataset

### Model evaluation

In [ ]:
train_loss, train_accu = fernet.evaluate(training_set)
test_loss, test_accu = fernet.evaluate(test_set)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))

In [ ]:
fernet.save_weights('fernet_bestweight.h5')

### Confusion Matrix and Classification on training set

In [ ]:
y_pred = model.predict(training_set)
y_pred = np.argmax(y_pred, axis=1)
class_labels = test_set.class_indices
class_labels = {v:k for k,v in class_labels.items()}

from sklearn.metrics import classification_report, confusion_matrix
cm_train = confusion_matrix(training_set.classes, y_pred)
print('Confusion Matrix')
print(cm_train)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(training_set.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_train, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)

### Confusion Matrix and Classification on test set

In [ ]:
y_pred = fernet.predict(test_set)
y_pred = np.argmax(y_pred, axis=1)
class_labels = test_set.class_indices
class_labels = {v:k for k,v in class_labels.items()}

#from sklearn.metrics import classification_report, confusion_matrix
cm_test = confusion_matrix(test_set.classes, y_pred)
print('Confusion Matrix')
print(cm_test)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(test_set.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_test, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)

In [ ]:
!